### To understand the library for this simple task of classification

taken from https://www.kdnuggets.com/2017/03/email-spam-filtering-an-implementation-with-python-and-scikit-learn.html


1. Preparing the text data.

2. Creating word dictionary.

3. Feature extraction process

4. Training the classifier


 training set: 702
 test set:260
 
 Cleanin already done
 * The emails in Ling-spam corpus have been already preprocessed in the following ways:
 - Removal of stop words 
 - Lemmatization
 
 
 Stil need to remove punctuation, special characters. we will remove such words after creating a dictionary.

#### Creating word dictionary

Mails look like : 

Subject: posting

hi , ' m work phonetics project modern irish ' m hard source . anyone recommend book article english ? ' , specifically interest palatal ( slender ) consonant , work helpful too . thank ! laurel sutton ( sutton @ garnet . berkeley . edu

In [6]:
import os
import collections as c

def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    
    dictionary = c.Counter(all_words)
    # Paste code for non-word removal here(code snippet is given below) 
    list_to_remove = dictionary.keys()
    for item in list(list_to_remove):
        if item.isalpha() == False or len(item) == 1: 
            dictionary.pop(item)
            
    #chosen 3000 most frequently used words in the dictionary.
    dictionary = dictionary.most_common(3000)
    return dictionary

In [7]:
x = c.Counter(["a", "b", "a", "c", "a", "a"])
y = x.keys()
list(y)

['a', 'b', 'c']

In [8]:
dictionary = make_Dictionary("train-mails")
print(dictionary[1:10])

[('address', 1293), ('report', 1216), ('mail', 1127), ('send', 1079), ('language', 1072), ('email', 1051), ('program', 1001), ('our', 987), ('list', 935)]


### 3. Feature extraction process.

The below python code will generate a feature vector matrix whose rows denote 700 files of training set and columns denote 3000 words of dictionary. The value at index ‘ij’ will be the number of occurrences of jth word of dictionary in ith file.


In [9]:
def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix

### Training the model


In [22]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix

# Create a dictionary of words with its frequency
train_dir = 'train-mails'
dictionary = make_Dictionary(train_dir)


# Prepare feature vectors per training mail and its labels
train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir)
print(train_matrix.shape)

(702, 3000)


In [23]:
train_matrix[1:10, 1:10]

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  2.,  4.,  0.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  2.,  0.,  0.]])

In [24]:
len(train_matrix[1,: ])

3000

In [25]:
# Training SVM and Naive bayes classifier

model1 = MultinomialNB()
#model2 = LinearSVC()
model1.fit(train_matrix,train_labels)
#model2.fit(train_matrix,train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
model1.classes_
model1.class_count_
model1.coef_

array([[-4.33163167, -4.47762944, -4.43418212, ..., -9.5581461 ,
        -9.42461471, -9.5581461 ]])

In [27]:
len(model1.coef_)

1

In [28]:
# Test the unseen mails for Spam
test_dir = 'test-mails'
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1
result1 = model1.predict(test_matrix)
#result2 = model2.predict(test_matrix)
print(confusion_matrix(test_labels,result1))
#print confusion_matrix(test_labels,result2)

[[129   1]
 [  9 121]]


In [34]:
cm = confusion_matrix(test_labels,result1)
(cm[0,0]+ cm[1,1])/sum(sum(cm))

0.96153846153846156

In [33]:
print(sum(sum(cm)))

260
